In [1]:
from google.colab import drive 
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import RegexpTokenizer as regextoken
from nltk.corpus import stopwords
from nltk import FreqDist, bigrams, trigrams
from nltk import WordNetLemmatizer
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import  make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import gensim
from gensim.models.keyedvectors import KeyedVectors
import re
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, Embedding, Conv1D, MaxPooling1D, Dropout, LSTM, GRU
from keras.regularizers import l1, l2
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")
plt.style.use('dark_background')


In [13]:
import pandas as pd
import numpy as np
from pandas import DataFrame

zomato=pd.read_csv('/content/drive/MyDrive/zomato.csv') #read csv
zomato = zomato[pd.notnull(zomato['cuisines'])]
zomato = zomato[pd.notnull(zomato['dish_liked'])]
zomato = zomato[pd.notnull(zomato['rest_type'])]
zomato = zomato[pd.notnull(zomato['location'])]
zomato = zomato[pd.notnull(zomato['rate'])]
zomato = zomato[pd.notnull(zomato['approx_cost(for two people)'])]
zomato['rate'] = zomato['rate'].str.replace(' ', '')
zomato = zomato[zomato['rate'] != 'NEW']
zomato = zomato[zomato['rate'] != '-']
zomato['approx_cost(for two people)'] = zomato['approx_cost(for two people)'].str.replace(',', '')
zomato['cuisines'] = zomato['cuisines'].str.replace(',', '')
zomato['dish_liked'] = zomato['dish_liked'].str.replace(',', '')
zomato['rest_type'] = zomato['rest_type'].str.replace(',', '')
zomato['location'] = zomato['location'].str.replace(',', '')
zomato['approx_cost(for two people)'].replace(np.NaN, zomato['approx_cost(for two people)'].median())
zomato = zomato.rename(columns={"approx_cost(for two people)":"cost_for_2"})
zomato.reset_index(inplace = True)
zomato.drop(['url', 'cost_for_2', 'index','address','name','online_order','book_table','votes','phone','reviews_list','menu_item','listed_in(type)','listed_in(city)'], inplace = True,axis=1)
zomato.shape

(23259, 5)

In [14]:
zomato.head()

,rate,location,rest_type,dish_liked,cuisines
0,4.1/5,Banashankari,Casual Dining,Pasta Lunch Buffet Masala Papad Paneer Lajawab...,North Indian Mughlai Chinese
1,4.1/5,Banashankari,Casual Dining,Momos Lunch Buffet Chocolate Nirvana Thai Gree...,Chinese North Indian Thai
2,3.8/5,Banashankari,Cafe Casual Dining,Churros Cannelloni Minestrone Soup Hot Chocola...,Cafe Mexican Italian
3,3.7/5,Banashankari,Quick Bites,Masala Dosa,South Indian North Indian
4,3.8/5,Basavanagudi,Casual Dining,Panipuri Gol Gappe,North Indian Rajasthani


In [15]:
zomato.to_csv("zomato_preprocessed.csv", index = False)

In [17]:
!cat  zomato_preprocessed.csv

Streaming output truncated to the last 5000 lines.
4.3/5,Brookefield,Cafe Bakery,Tea Pizza New York Cheesecake Hot Chocolate Sandwiches Sunday Brunch,Cafe Bakery Fast Food
4.6/5,Marathahalli,Casual Dining Cafe,Ricotta Ravioli Pasta Bbq Chicken Pizza Butter Chicken Pizza Farmhouse Pizza Mojito Barbeque Chicken Pizza,Pizza Cafe Italian
3.9/5,Marathahalli,Cafe,Cheesecake Pizza,Cafe Italian
2.8/5,Marathahalli,Cafe,Ginger Tea Poha Paratha Chai Chaai,Cafe Tea Beverages
3.6/5,Marathahalli,Cafe,Chicken Sandwich,Cafe Beverages Continental North Indian Chinese
4.1/5,Marathahalli,Cafe,Burgers Fries Coffee Mojito Prawn American Cheese Burger Mocktails,Cafe Burger Fast Food
3.6/5,Marathahalli,Cafe,Coffee Cappuccino Sandwiches Chocolate Fantasy Tea Nachos Brownie,Cafe
4.1/5,Brookefield,Cafe,Tea Chai Nachos Poha Veg Sandwich Bombay Sandwich Sandwiches,Cafe Street Food
4.0/5,Brookefield,Cafe,Burgers Pasta Noodles Shawarma Tea Cappuccino Coffee,Burger Juices Beverages Fast Food Cafe Chinese Coffee
3.2/